# ABM Traffic Flow Thesis Project
**By Joep Saris - s1094356 - 2025**


#### Code is divided into:
1. Class and Function setup
2. Showing features and running simulations

**Class and Function setup**
1. Imports

In [1]:
from abmax.structs import *
from abmax.functions import *
import jax.numpy as jnp
import jax.random as random
import jax
from flax import struct

# Imports for non-static / non-jax parts of code
import numpy as np
from collections import deque
import random as rand

# Richer representation for testing (visualizing coords in traffic problem)
from rich.console import Console
from rich.table import Table

2. Class code

In [67]:
@staticmethod
class Cell:
    """Represents a single cell on the map."""

    def __init__(self, coords, lane, entry=False, exit=False, accessible=True, heading_direction=2, car_list=None, toggle_group=None):
        self.coords = coords # (Y, X) coordinates
        self.entry = entry # False for no entry , True for entry position
        self.exit = exit # False for not exit, True for exit position
        self.accessible = accessible  # False for inaccessible, True for accessible
        self.heading_direction = heading_direction # 0 for down, 1 for left, 2 for up, 3 for right
        self.lane = lane # 1 for left border, 0 for middle lanes, 2 for right border, 3 for solo lanes
        self.car_list = deque(car_list or [], maxlen=2) # deque of cars in Cell
        self.toggle_group = toggle_group  # Group for traffic lights
    
    # Updating Cell
    def change_car_list(self, new_car_list):
        return Cell(coords=self.coords, lane=self.lane, entry=self.entry, exit=self.exit, accessible=self.accessible, heading_direction=self.heading_direction, car_list=new_car_list, toggle_group=self.toggle_group)
    def change_direction_and_lane(self, new_dir, new_lane):
        return Cell(coords=self.coords, lane=new_lane, entry=self.entry, exit=self.exit, accessible=self.accessible, heading_direction=new_dir, car_list=self.car_list, toggle_group=self.toggle_group)
    def toggle_cell_accessibility(self):
        toggled_accessibility = not self.accessible
        return Cell(coords=self.coords, lane=self.lane, entry=self.entry, exit=self.exit, accessible=toggled_accessibility, heading_direction=self.heading_direction, car_list=self.car_list, toggle_group=self.toggle_group)

    # Misc
    def surrounding_cells(self, road, radius=1):
        y, x = self.coords
        coords = []
        for i in range(radius+1):
            for j in range(radius+1):
                coords.append((y-i, x-j))
                coords.append((y-i, x+j))
                coords.append((y+i, x+j))
                coords.append((y+i, x-j))
        coords = list(set(coords))
        coords.remove(self.coords)
        print(coords)
        return road.remove_invalid_coords(coords)
    # Representation
    def __str__(self):
        return str(self.car_list[0]) if self.car_list else 'X' if not self.accessible else 'S' if self.entry else 'D' if self.exit else str(self.lane)
    def tooltip(self):
        return f"{self.coords[0]}-{self.coords[1]}"

# Navigation setup
DIRECTIONS = ((1, 0), (0, -1), (-1, 0), (0, 1))  # Down, Left, Up, Right
ARROWS = ("↓", "↲", "↳", "←", "↰", "↲", "↑", "↱", "↰", "→", "↳", "↱", "↶") # "↶" for U

class Road:
    """Represents the map and runs the simulation."""

    # Setup
    def __init__(self, grid):
        self.grid = grid
        self.start_coords = [(y, x) for y in range(grid.shape[0]) for x in range(grid.shape[1]) if grid[y, x].entry] 
        self.exit_coords = [(y, x) for y in range(grid.shape[0]) for x in range(grid.shape[1]) if grid[y, x].exit]
        self.toggle_group_dict = self.find_toggle_group_dict()
    def find_toggle_group_dict(self):
        """Creates a dictionary mapping toggle_group values to lists of corresponding cell coordinates."""
        toggle_group_coords = {}
        for row in self.grid:
            for cell in row:
                toggle_group_coords.setdefault(cell.toggle_group, []).append(cell.coords)
        return toggle_group_coords
    def grid_add_cars_at(self, car_coords: list, car_destinations: list = None):
        for i in range(len(car_coords)):
            coord = car_coords[i]
            if (0 <= coord[0] < self.grid.shape[0]) and (0 <= coord[1] < self.grid.shape[1]): # only valid coords should get Car objects
                if car_destinations:
                    car = Car(cell=self.grid[coord], car_id=i, road=self, destination=car_destinations[i%len(car_destinations)])
                else:
                    car = Car(cell=self.grid[coord], car_id=i, road=self)
                    
                updated_car_list = self.grid[coord].car_list.copy() # copy car list
                updated_car_list.append(car) # update car list to have car
                self.grid[coord] = self.grid[coord].change_car_list(new_car_list=updated_car_list) # add modified Cell to grid
    def no_light_intersection(self, intersection_base, intersection_length):
        lanes = intersection_length / 2
        for i in range(intersection_length):
            for j in range(intersection_length):
                # Dividing into quadrants top_left = 0, top_right = 1, bottom_right = 2, bottom_left = 3 (synonomous with heading_direction: down, left, up, right)
                check_Y = jnp.floor(i / lanes) 
                check_X = jnp.floor(j / lanes)
                lane = 0 # Every value is 0, except the leftmost column for each relative position
                if check_Y and check_X:
                    quad = 2 # Bottom right quadrant, goes up                               0 0 1   0 0 0   | Quadrant's zero rows/columns, examplified with length=6    1 < 1  | Minimal length = 2
                    if j == lanes: # relative left for upward direction is global left      0 0 1 < 0 0 0   | Top_left     (0) = quadrant's right column (lanes-1)       v   ^  |
                        lane = 1 #                                                          0 0 1   1 1 1   | Top_right    (1) = quadrant's bottom row (lanes-1)         1 > 1  |
                elif check_Y: #                                                               v       ^
                    quad = 3 # Bottom left quadrant, goes right                             1 1 1   1 0 0   | Bottom_left  (3) = quadrant's top row (lanes)
                    if i == lanes: # relative left for rightward direction is global up     0 0 0 > 1 0 0   | Bottom_right (2) = quadrant's left column (lanes)
                        lane = 1 #                                                          0 0 0   1 0 0   | Proud of this design <3
                elif check_X:
                    quad = 1 # Top right quadrant, goes left
                    if i == (lanes-1): # relative left for leftward direction is global down
                        lane = 1
                else:
                    quad = 0 # Top left quadrant, goes down
                    if j == (lanes-1): # relative left for downward direction is global right
                        lane = 1
                self.grid[intersection_base[0] + i, intersection_base[1] + j] = self.grid[intersection_base[0] + i, intersection_base[1] + j].change_direction_and_lane(new_dir=quad, new_lane=lane)

    # Simulation
    def coord_plus_direction(self, coord: tuple, direction: int):
        new_coord = (coord[0] + DIRECTIONS[direction % 4][0], coord[1] + DIRECTIONS[direction % 4][1])
        if (0 < new_coord[0] <= self.grid.shape[0]) and (0 < new_coord[1] <= self.grid.shape[1]):
            return new_coord
        return None
    def remove_invalid_coords(self, coords: list):
        valid_coords = []
        for coord in coords:
            y, x = coord
            if (0 <= y < self.grid.shape[0]) and (0 <= x < self.grid.shape[1]):
                valid_coords.append(coord)
        return valid_coords
    def first_car_moves(self, cell_a: Cell):
        if cell_a.car_list:
            first_car = cell_a.car_list[0]
            cell_b = self.grid[first_car.next_move[0]]
            if cell_b.accessible:
                new_car_list_a = cell_a.car_list.copy()
                new_car_list_a.remove(first_car)
                self.grid[cell_a.coords] = Cell(coords=cell_a.coords, lane=cell_a.lane, entry=cell_a.entry, exit=cell_a.exit, accessible=cell_a.accessible, heading_direction=cell_a.heading_direction, car_list=new_car_list_a, toggle_group=cell_a.toggle_group)
                new_car = Car(cell=cell_b, car_id=first_car.car_id, road=self, destination=first_car.destination, uturn=first_car.uturn)
                return True, new_car
        return False, None
    def toggle_traffic_lights(self, group: int):
        """Switches the state of traffic light cells every 4 timesteps."""
        toggle_group_members = self.toggle_group_dict[group] # Finds traffic light group's coords
        updated_grid = self.grid.copy()
        for coord in toggle_group_members: # Every Cell in group gets their accessibility toggled
            updated_grid[coord] = self.grid[coord].toggle_cell_accessibility()
        self.grid = updated_grid
    def step_grid(self, steps=10):
        print("      start \n", self)
        for i in range(steps):
            new_cars = []
            for row in self.grid:
                for cell in row:
                    if cell.car_list:
                        moved, new_car = self.first_car_moves(cell)
                        if new_car:
                            new_cars.append(new_car)
            for car in new_cars:
                cell = self.grid[car.cell.coords]
                cell.car_list.append(car)
            print(f"   timestep {i+1} \n", self)
            self.toggle_traffic_lights(1)

    # Representation
    def __str__(self):
        string_reprs = [[str(cell) for cell in row] for row in self.grid]
        max_width = max(len(cell_str) for row in string_reprs for cell_str in row)
        return "\n".join([
            " ".join(f"{cell_str:>{max_width}}" for cell_str in row)
            for row in string_reprs
        ])
    def display(self, expand=False):
        console = Console()
        table = Table(show_header=False, show_lines=True, padding=(-2,-1), expand=True)

        for row in self.grid:
            table.add_row(*[f"[bold]{str(cell):}[/bold]\n[dim]{cell.tooltip()}[/dim]" for cell in row])
        console.print(table)
    
@staticmethod
class Car:
    """Represents a car moving through the grid."""
    # Setup
    def __init__(self, cell: Cell, car_id: int, road: Road, traffic: object = None, destination: tuple = None, uturn: bool = True):
        self.cell: Cell = cell
        self.car_id: int = car_id
        self.uturn: bool = uturn
        self.destination = destination if destination else rand.choice(road.exit_coords)
        self.next_move: tuple = self.choose_random_next_move(road)
        #traffic.update_car(self)

    # Stepping
    def find_all_moves(self, cell: Cell, road: Road): # Finds options from specified cell
        """Find next coord for Car"""
        if cell is not None:
            if cell.coords == self.destination:  # If at destination, pick a random start location
                return list(map(lambda coord: (coord, "✓"), road.start_coords)) # Hocus pocus create list all mapped to "✓" value
            forward_coord = (road.coord_plus_direction(cell.coords, cell.heading_direction))
            if forward_coord is not None:
                forward = road.grid[forward_coord]
                if (forward.accessible or (not forward.accessible and forward.toggle_group)):
                        possible_moves = [((forward_coord, ARROWS[cell.heading_direction*3]))]
                # Rightward lane switch
                if cell.lane in {0, 1}:
                    right_coord = (road.coord_plus_direction(forward_coord, cell.heading_direction + 1))
                    if right_coord is not None:
                        right = road.grid[right_coord]
                        if (right.accessible or (not right.accessible and right.toggle_group)):
                                possible_moves.append((right_coord, ARROWS[cell.heading_direction*3 + 1]))
                # Leftward lane switch
                if cell.lane in {0, 2}:
                    left_coord = (road.coord_plus_direction(forward_coord, cell.heading_direction - 1))
                    if left_coord is not None:
                        left = road.grid[left_coord]
                        if (left.accessible or (not left.accessible and left.toggle_group)):
                                possible_moves.append((left_coord, ARROWS[cell.heading_direction*3 + 2]))
                # U-turn lane switch (crossing over to other road side going opposite direction)
                if cell.lane in {-1, 1} and not (cell.exit or cell.entry):
                    if self.uturn:
                        left_coord = (road.coord_plus_direction(forward_coord, cell.heading_direction - 1))
                        if left_coord is not None:
                            left = road.grid[left_coord]
                            if left.lane in {-1, 1} and (left.accessible or (not left.accessible and left.toggle_group)):
                                    possible_moves.append((left_coord, ARROWS[-1]))
                return possible_moves
            # Return original cell if no other move is possible
            return [(cell.coords, ".")]
    def choose_random_next_move(self, road: Road): # Finds options from current cell and chooses one randomly
        if self.cell.exit:  # If at exit location, pick a random start location
            return rand.choice(list(map(lambda coord: (coord, "✓"), road.start_coords))) # Hocus pocus create list all mapped to "✓" value
        else:
            return rand.choice(self.find_all_moves(self.cell, road))

    # Planning
    def shortest_path_to_destination(self, road): # Finds the path to destination with least Cells crossed
        start = self.cell.coords
        queue = deque([(start, [])])
        visited = [start]
        while queue:
            current, path = queue.popleft()
            path = path + [current]  
            if current == self.destination:
                return path
            current_cell = road.grid[current]
            for neighbor in self.find_all_moves(cell=current_cell, road=road):
                neighbor_coord = neighbor[0]
                if neighbor_coord is not None:
                    if neighbor_coord not in visited:
                        visited.append(neighbor_coord)
                        queue.append((neighbor_coord, path))
        return None

    # Representation
    def __str__(self): 
        return str(self.next_move[1])
    
"""
class Traffic:
    Represents all cars in the simulation, either active or inactive.
    def __init__(self, car_list=None):
        self.car_list = car_list or []

    def update_car(self, new_car: Car):
        Replaces the car with the same car_id in car_list. Returns False if not found.
        for i, car in enumerate(self.car_list):
            if car.car_id == new_car.car_id:
                self.car_list[i] = new_car
                print(f"Car {car.car_id} got updated")
                return True
        print(f"Car {new_car.car_id} wasn't found, so it got added")
        self.add_car(new_car)
        return False

    def add_car(self, car: Car):
        Adds a new car to the traffic system.
        self.car_list.append(car)
"""

'\nclass Traffic:\n    Represents all cars in the simulation, either active or inactive.\n    def __init__(self, car_list=None):\n        self.car_list = car_list or []\n\n    def update_car(self, new_car: Car):\n        Replaces the car with the same car_id in car_list. Returns False if not found.\n        for i, car in enumerate(self.car_list):\n            if car.car_id == new_car.car_id:\n                self.car_list[i] = new_car\n                print(f"Car {car.car_id} got updated")\n                return True\n        print(f"Car {new_car.car_id} wasn\'t found, so it got added")\n        self.add_car(new_car)\n        return False\n\n    def add_car(self, car: Car):\n        Adds a new car to the traffic system.\n        self.car_list.append(car)\n'

3. Road grid construction

In [10]:
# Road work ahead?
def create_road(dims: tuple): # Creates small one way road object in specified dimensions (Y, X), with toggle groups at the top row
        grid = np.empty(dims, dtype=object)
        for y in range(dims[0]):
            exit = True if y == 0 else False
            entry = True if y == dims[0] - 1 else False          

            for x in range(dims[1]):
                lane = 1 if x == 0 else (2 if x == dims[1] - 1 else 0)
                if exit: # Setting exit cells to toggle group 1
                    grid[y, x] = Cell(coords=(y, x), lane=lane, entry=entry, exit=exit, toggle_group=1)
                else:
                    grid[y, x] = Cell(coords=(y, x), lane=lane, entry=entry, exit=exit)
        return Road(grid)
def square_road(nr_junctions, nr_lanes, connection_length): # Creates square traffic system road
    """
    Sets up map as a square matrix of size based on parameters:
    nr_junctions: specifies the number of junctions in both horizontal and vertical axis (resultant grid has nr_junctions squared juctions)
    nr_lanes: specifies how many lanes a one way road has. The cells in these lanes get the appropriate lane value to ensure intended lane switching behavior
    connection_length: specifies how many Cells are between every intersection
    """
    two_lanes = 2 * nr_lanes
    map_length = nr_junctions * two_lanes + (nr_junctions + 1) * connection_length
    cell_list = np.empty((map_length, map_length), dtype=object)

    final_connection = nr_junctions*(connection_length + two_lanes)
    intersection_bases = []

    for base_Y in range(0, map_length, (connection_length + two_lanes)):
        if (base_Y == final_connection):
            lowest_connection = True
        else:
            lowest_connection = False

        for base_X in range(0, map_length, (connection_length + two_lanes)):
            if (base_X == final_connection):
                rightmost_connection = True
            else:
                rightmost_connection = False


            # Setting connection Cells
            if not lowest_connection: # Adding horizontal connection
                for lane_offset in range(two_lanes):
                    for connection_offset in range(connection_length):
                        X = base_X + connection_offset

                        # Setting lane
                        if nr_lanes > 1:
                            if lane_offset == 0 or lane_offset == two_lanes-1:
                                lane = 2 # Right border lane
                            elif lane_offset == nr_lanes-1 or lane_offset == nr_lanes:
                                    lane = 1 # Left border lane
                            else:
                                lane = 0 # Middle lane
                        else:
                            lane = -1 # Solo lane
                        
                        # Seperating the two driving direction connections
                        if lane_offset < nr_lanes: # Top half connection
                            direction = 1 # Global heading_direction is left

                            # Setting cell_type                            
                            if X == 0: # If in first column, it is an exit cell (2)
                                entry = False
                                exit = True
                            elif X == map_length-1: # If in last column, it is an entry cell (1)
                                entry = True
                                exit = False 
                            else: # In any other case, it is a regular road cell (0)
                                entry = False
                                exit = False

                        else: # Bottom half connection
                            direction = 3 # Global heading_direction is right
    
                            # Setting cell_type 
                            if X == 0: # If in first column, it is an entry cell (1)
                                entry = True
                                exit = False 
                            elif X == map_length-1: # If in last column, it is an exit cell (2)
                                entry = False
                                exit = True
                            else: # In any other case, it is a regular road cell (0)
                                entry = False
                                exit = False

                        cell_list[base_Y + connection_length + lane_offset, X] = Cell(coords=(base_Y + connection_length + lane_offset, X), lane=lane, entry=entry, exit=exit, heading_direction=direction)
            
            if not rightmost_connection: # Adding vertical connection
                for connection_offset in range(connection_length):
                    for lane_offset in range(two_lanes):
                        Y = base_Y + connection_offset
                        # Setting lane
                        if nr_lanes > 1:
                            if lane_offset == 0 or lane_offset == two_lanes-1:
                                lane = 2 # Right border lane
                            elif lane_offset == nr_lanes-1 or lane_offset == nr_lanes:
                                    lane = 1 # Left border lane
                            else:
                                lane = 0 # Middle lane
                        else:
                            lane = -1 # Solo lane
                        
                        # Seperating the two driving direction connections
                        if lane_offset < nr_lanes: # Left half connection
                            direction = 0 # Global heading_direction is down

                            # Setting cell_type
                            if Y == 0: # If in first row, it is an entry cell (1)
                                entry = True
                                exit = False 
                            elif Y == map_length-1: # If in last row, it is an exit cell (2)
                                entry = False
                                exit = True
                            else: # In any other case, it is a regular road cell (0)
                                entry = False
                                exit = False

                        else: # Bottom half connection
                            direction = 2 # Global heading_direction is up
    
                            # Setting cell_type 
                            if Y == 0: # If in first row, it is an exit cell (2)
                                entry = False
                                exit = True
                            elif Y == map_length-1: # If in last row, it is an entry cell (1)
                                entry = True
                                exit = False
                            else: # In any other case, it is a regular road cell (0)
                                entry = False
                                exit = False
                        
                        cell_list[Y, base_X + connection_length + lane_offset] = Cell(coords=(Y, base_X + connection_length + lane_offset), lane=lane, entry=entry, exit=exit, heading_direction=direction)

            # Setting inaccessible Cells
            for i in range(connection_length):
                for j in range(connection_length):
                    cell_list[base_Y + i, base_X + j] = Cell(coords=(base_Y + i, base_X + j), accessible=False, heading_direction=-1, lane=-1)

            # Setting intersection Cells, initialized with heading_direction and lane being -1, to be decorated later using Map.no_light_intersection(intersection_base, intersection_length)
            if not rightmost_connection and not lowest_connection:
                intersection_base_Y = base_Y + connection_length
                intersection_base_X = base_X + connection_length
                intersection_bases.append((intersection_base_Y, intersection_base_X))
                for i in range (two_lanes):
                    for j in range (two_lanes):
                        cell_list[intersection_base_Y + i, intersection_base_X + j] = Cell(coords=(intersection_base_Y + i, intersection_base_X + j), lane=lane, heading_direction=direction)
    road = Road(cell_list)
    for intersection_base in intersection_bases:
        road.no_light_intersection(intersection_base=intersection_base, intersection_length=two_lanes)
    return road

**Showing above functions in action**
1. Traffic simulation for parallel movement

In [56]:
road: Road = create_road((7, 3))
road.display()
road.grid_add_cars_at([(5, 1), (5, 0), (2, 2), (4, 1)])
road.step_grid(steps=15)



┌─────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────────┐
│D                                    │D                                    │D                                    │
│0-0                                  │0-1                                  │0-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│1-0                                  │1-1                                  │1-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│2-0                                  │2-1                                  │2-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│3-0                                  │3-1                                  │3-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│4-0                                  │4-1                                  │4-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│5-0                                  │5-1                                  │5-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│S                                    │S                                    │S                                    │
│6-0                                  │6-1                                  │6-2                                  │
└─────────────────────────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘

      start 
 D D D
1 0 2
1 0 ↰
1 0 2
1 ↱ 2
1 ↱ 2
S S S
   timestep 1 
 D D D
1 . 2
1 0 2
1 0 ↑
1 0 ↑
1 0 2
S S S
   timestep 2 
 X X X
1 . 2
1 0 ↑
1 0 ↰
1 0 2
1 0 2
S S S
   timestep 3 
 D D D
1 . .
1 ↱ 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 4 
 X X X
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 5 
 D D D
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 6 
 X X X
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 7 
 D D D
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 8 
 X X X
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 9 
 D D D
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 10 
 X X X
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 11 
 D D D
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 12 
 X X X
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 13 
 D D D
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 14 
 X X X
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S
   timestep 15 
 D D D
1 . .
1 0 2
1 0 2
1 0 2
1 0 2
S S S


2. Shortest path in complex road structure - using U-turn

In [47]:
# Creating a large 
tester = square_road(nr_junctions=2, nr_lanes=3, connection_length=3)
print(tester)
tester.display()

tester.grid_add_cars_at([(7, 9)], [(4, 9)])
hidden_car = tester.grid[(7, 9)].car_list[0]
print(hidden_car.shortest_path_to_destination(tester))

X X X S S S D D D X X X S S S D D D X X X
X X X 2 0 1 1 0 2 X X X 2 0 1 1 0 2 X X X
X X X 2 0 1 1 0 2 X X X 2 0 1 1 0 2 X X X
D 2 2 0 0 1 0 0 0 2 2 2 0 0 1 0 0 0 2 2 S
D 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 S
D 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 S
S 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 D
S 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 D
S 2 2 0 0 0 1 0 0 2 2 2 0 0 0 1 0 0 2 2 D
X X X 2 0 1 1 0 2 X X X 2 0 1 1 0 2 X X X
X X X 2 0 1 1 0 2 X X X 2 0 1 1 0 2 X X X
X X X 2 0 1 1 0 2 X X X 2 0 1 1 0 2 X X X
D 2 2 0 0 1 0 0 0 2 2 2 0 0 1 0 0 0 2 2 S
D 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 S
D 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 S
S 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 D
S 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 D
S 2 2 0 0 0 1 0 0 2 2 2 0 0 0 1 0 0 2 2 D
X X X 2 0 1 1 0 2 X X X 2 0 1 1 0 2 X X X
X X X 2 0 1 1 0 2 X X X 2 0 1 1 0 2 X X X
X X X D D D S S S X X X D D D S S S X X X


┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬────┬────┐
│X   │X   │X   │S   │S   │S   │D   │D   │D   │X   │X    │X    │S    │S    │S    │D    │D    │D    │X    │X   │X   │
│0-0 │0-1 │0-2 │0-3 │0-4 │0-5 │0-6 │0-7 │0-8 │0-9 │0-10 │0-11 │0-12 │0-13 │0-14 │0-15 │0-16 │0-17 │0-18 │0-19│0-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│X   │X   │X   │2   │0   │1   │1   │0   │2   │X   │X    │X    │2    │0    │1    │1    │0    │2    │X    │X   │X   │
│1-0 │1-1 │1-2 │1-3 │1-4 │1-5 │1-6 │1-7 │1-8 │1-9 │1-10 │1-11 │1-12 │1-13 │1-14 │1-15 │1-16 │1-17 │1-18 │1-19│1-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│X   │X   │X   │2   │0   │1   │1   │0   │2   │X   │X    │X    │2    │0    │1    │1    │0    │2    │X    │X   │X   │
│2-0 │2-1 │2-2 │2-3 │2-4 │2-5 │2-6 │2-7 │2-8 │2-9 │2-10 │2-11 │2-12 │2-13 │2-14 │2-15 │2-16 │2-17 │2-18 │2-19│2-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│D   │2   │2   │0   │0   │1   │0   │0   │0   │2   │2    │2    │0    │0    │1    │0    │0    │0    │2    │2   │S   │
│3-0 │3-1 │3-2 │3-3 │3-4 │3-5 │3-6 │3-7 │3-8 │3-9 │3-10 │3-11 │3-12 │3-13 │3-14 │3-15 │3-16 │3-17 │3-18 │3-19│3-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│D   │0   │0   │0   │0   │1   │0   │0   │0   │0   │0    │0    │0    │0    │1    │0    │0    │0    │0    │0   │S   │
│4-0 │4-1 │4-2 │4-3 │4-4 │4-5 │4-6 │4-7 │4-8 │4-9 │4-10 │4-11 │4-12 │4-13 │4-14 │4-15 │4-16 │4-17 │4-18 │4-19│4-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│D   │1   │1   │0   │0   │1   │1   │1   │1   │1   │1    │1    │0    │0    │1    │1    │1    │1    │1    │1   │S   │
│5-0 │5-1 │5-2 │5-3 │5-4 │5-5 │5-6 │5-7 │5-8 │5-9 │5-10 │5-11 │5-12 │5-13 │5-14 │5-15 │5-16 │5-17 │5-18 │5-19│5-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│S   │1   │1   │1   │1   │1   │1   │0   │0   │1   │1    │1    │1    │1    │1    │1    │0    │0    │1    │1   │D   │
│6-0 │6-1 │6-2 │6-3 │6-4 │6-5 │6-6 │6-7 │6-8 │6-9 │6-10 │6-11 │6-12 │6-13 │6-14 │6-15 │6-16 │6-17 │6-18 │6-19│6-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│S   │0   │0   │0   │0   │0   │1   │0   │0   │0   │0    │0    │0    │0    │0    │1    │0    │0    │0    │0   │D   │
│7-0 │7-1 │7-2 │7-3 │7-4 │7-5 │7-6 │7-7 │7-8 │7-9 │7-10 │7-11 │7-12 │7-13 │7-14 │7-15 │7-16 │7-17 │7-18 │7-19│7-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│S   │2   │2   │0   │0   │0   │1   │0   │0   │2   │2    │2    │0    │0    │0    │1    │0    │0    │2    │2   │D   │
│8-0 │8-1 │8-2 │8-3 │8-4 │8-5 │8-6 │8-7 │8-8 │8-9 │8-10 │8-11 │8-12 │8-13 │8-14 │8-15 │8-16 │8-17 │8-18 │8-19│8-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│X   │X   │X   │2   │0   │1   │1   │0   │2   │X   │X    │X    │2    │0    │1    │1    │0    │2    │X    │X   │X   │
│9-0 │9-1 │9-2 │9-3 │9-4 │9-5 │9-6 │9-7 │9-8 │9-9 │9-10 │9-11 │9-12 │9-13 │9-14 │9-15 │9-16 │9-17 │9-18 │9-19│9-20│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│X   │X   │X   │2   │0   │1   │1   │0   │2   │X   │X    │X    │2    │0    │1    │1    │0    │2    │X    │X   │X   │
│10-0│10-1│10-2│10-3│10-4│10-5│10-6│10-7│10-8│10-9│10-10│10-11│10-12│10-13│10-14│10-15│10-16│10-17│10-18│10-1│10-2│
├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼────┤
│X   │X   │X   │2   │0   │1   │1   │0   │2   │X   │X    

[(7, 9), (6, 10), (5, 11), (5, 10), (4, 9)]


3. Coders graveyard

In [69]:
new_road = create_road((7, 3))
new_road.display()

marcel: Cell = new_road.grid[(0, 0)]
marcel_surrounds = marcel.surrounding_cells(new_road)
print(marcel_surrounds)


┌─────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────────┐
│D                                    │D                                    │D                                    │
│0-0                                  │0-1                                  │0-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│1-0                                  │1-1                                  │1-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│2-0                                  │2-1                                  │2-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│3-0                                  │3-1                                  │3-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│4-0                                  │4-1                                  │4-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│1                                    │0                                    │2                                    │
│5-0                                  │5-1                                  │5-2                                  │
├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────────┤
│S                                    │S                                    │S                                    │
│6-0                                  │6-1                                  │6-2                                  │
└─────────────────────────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘

[(0, 1), (-1, -1), (-1, 1), (1, 1), (1, -1), (-1, 0), (1, 0), (0, -1)]
[(0, 1), (1, 1), (1, 0)]


In [31]:
for i in range (1, 2):
    print("this happens once")

this happens once


In [ ]:
car_list = np.array([None, None])

if car_list.any():
    print("we are not empty!!")
car_list[0] = 42
car_list[1] = 58
if car_list.any():
    print("we are not empty")

if car_list.all():
    print("you cannot go here")
else:
    if car_list[0] is None:
        print("you were put in slot one")
    else:
        print("you were put in slot two")


car_list[0] = car_list[1]  # Shift second item to first
car_list[1] = None

if car_list.all():
    print("you cannot go here")
else:
    if car_list[0] is None:
        print("you were put in slot one")
    else:
        print("you were put in slot two")

we are not empty
you cannot go here
you were put in slot two


In [ ]:
# Integration artifacts

'''
road function
def inaccessible_intersection(self, intersection_base, intersection_length):
        """Sets red_light to all Cells in intersection."""
        for i in range(intersection_length):
            for j in range(intersection_length):
                self.grid[intersection_base[0] + i, intersection_base[1] + j] = self.grid[intersection_base[0] + i, intersection_base[1] + j].toggle_cell_accessibility()

                
'''

# Where the JAXcar goes
@struct.dataclass
class Car(Agent):
    """
    state :
    current location (Y, X),
    direction_of_heading: 0: up, 1: down, 2: left, 3: right
    chaos: 0: no chaos, 1: chaos
    speed: 0: stopped, 1: moving
    
    parameters:
    map: map of the environment
    car_length: length of the car
    destination: destination of the car (Y, X)
    """
    @staticmethod
    def create_agent(type, params, id, active_state, key, policy):
        key, subkey = random.split(key)
    
        def create_active_agent():
            subkey, *create_keys = random.split(subkey, 3) #3 will change based on need
            map = params['map']
            start_locations = params['start_locations']
            destinations = params['destinations']
            start = start_locations[jax.random.randint(create_keys[0], 1, 1, len(start_locations))[0]]
            destination = destinations[jax.random.randint(create_keys[1], 1, 1, len(destinations))[0]]
            state_content = {'current_location': start, 'direction_of_heading': 0, 'chaos': 0, 'speed': 0}


            
        
        def create_inactive_agent():
            pass
        
        def get_surroundings():
            pass
            """try :
                x, y = state_content['current_location']
                return [map[x, y+1], map[x+1, y+1], map[x+1, y], map[x, y-1], map[x-1, y-1], map[x-1, y], map[x-1, y+1], map[x+1, y-1]]
            return"""
            
 
        def create_inactive_agent():
            pass



    """
    Represents a car in the environment.
    """
    """
    coords: jnp.array # [Y, X] coordinates of cell in Map
    direction: int # -1: none, 0: down, 1: left, 2: up, 3: right
    chaos: int # not implemented
    speed: int # number of squares a car travels in a single tick 
    destination: jnp.array # [Y, X] coordinates of destination
    uturn: bool # whether or not a NonJax_Car is allowed on connection_road
    """